# **Installs**



In [8]:
!pip install camel-tools'

/bin/bash: -c: line 1: unexpected EOF while looking for matching `''
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [9]:
!pip install torch

In [10]:
!pip install datasets

In [11]:
!pip install --upgrade jax jaxlib'

/bin/bash: -c: line 1: unexpected EOF while looking for matching `''
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [12]:
!pip install transformers[torch] #for prototypical training

In [13]:
!pip install --upgrade transformers

# **Imports**

In [7]:
#to access data
import xml.etree.ElementTree as ET
import os
import pandas as pd
from google.colab import drive
#for NER
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [14]:
import torch

In [15]:
from datasets import load_dataset
from datasets import Dataset
import re

In [16]:
import json #for Doccano

In [17]:
import torch #for prototypical networks
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import random

In [90]:
import json #for finetuning
import os
import random
from typing import List, Dict

# **Exploratory Data Analysis**

In [18]:
# Mount the Google Drive at /content/drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/AKG Research/levels0to3/Maroc-Primaire-0.xml'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Explore file structure

In [19]:
tree = ET.parse(file_path)
root = tree.getroot()

In [20]:
structure = []
def explore_element(element, depth=0):
    structure.append(f"{'  ' * depth}<{element.tag}>")
    for child in element:
        explore_element(child, depth + 1)

In [21]:
explore_element(root)
structure[:10]

['<book>',
 '  <text>',
 '    <segment_info>',
 '      <sentence>',
 '      <analysis>',
 '        <morph_feature_set>',
 '        <morph_feature_set>',
 '        <morph_feature_set>',
 '    <segment_info>',
 '      <sentence>']

In [22]:
folder_path = '/content/drive/MyDrive/AKG Research/levels0to3'
all_data = []

In [23]:
for filename in os.listdir(folder_path):

        print("reading file", filename)
        file_path = os.path.join(folder_path, filename)
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Extracting data for each sentence and its analysis
        for text in root.findall(".//text"):
            text_id = text.get('id')
            title = text.get('title')

            for segment in text.findall(".//segment_info"):
                sentence_id = segment.get('id')
                sentence = segment.find('sentence').text

                # Loop over morph_feature_set to get word-level features
                for morph in segment.findall(".//morph_feature_set"):
                    word = morph.get('word')
                    lemma = morph.get('lemma')
                    pos = morph.get('pos')
                    stem = morph.get('stem')
                    root_word = morph.get('root')

                    # Append extracted data to the list
                    all_data.append([filename, text_id, title, sentence_id, sentence, word, lemma, pos, stem, root_word])

reading file Maroc-Primaire-3.xml
reading file Maroc-Primaire-2-p2.xml
reading file Maroc-Primaire-1.xml
reading file Maroc-Primaire-2.xml
reading file Maroc-Primaire-0.xml


In [24]:
all_data[0]


['Maroc-Primaire-3.xml',
 '1',
 'أبر والدي',
 '1',
 'أبر والدي',
 'أبر',
 'أَبَرَ',
 'VePa',
 'أَبَر',
 'ءبر']

In [25]:
#DataFrame
columns = ['file', 'text_id', 'title', 'sentence_id', 'sentence', 'word', 'lemma', 'pos', 'stem', 'root']
df = pd.DataFrame(all_data, columns=columns)

In [26]:
df.head(10)

,file,text_id,title,sentence_id,sentence,word,lemma,pos,stem,root
0,Maroc-Primaire-3.xml,1,أبر والدي,1,أبر والدي,أبر,أَبَرَ,VePa,أَبَر,ءبر
1,Maroc-Primaire-3.xml,1,أبر والدي,1,أبر والدي,والدي,وَالِد,Sifa.S,وَالِد,ولد
2,Maroc-Primaire-3.xml,1,أبر والدي,2,كان الوقت مساء، وفي المساء يحلو لنا أن نلتقي ف...,كان,كَانَ,VePa nasikh,كَان,كون
3,Maroc-Primaire-3.xml,1,أبر والدي,2,كان الوقت مساء، وفي المساء يحلو لنا أن نلتقي ف...,الوقت,وَقْت,DNo,وَقْت,وقت
4,Maroc-Primaire-3.xml,1,أبر والدي,2,كان الوقت مساء، وفي المساء يحلو لنا أن نلتقي ف...,مساء,مَسَاء,No NounInfinit,مَسَاء,-
5,Maroc-Primaire-3.xml,1,أبر والدي,2,كان الوقت مساء، وفي المساء يحلو لنا أن نلتقي ف...,وفي,فِي,Conj P.Pr,فِي,-
6,Maroc-Primaire-3.xml,1,أبر والدي,2,كان الوقت مساء، وفي المساء يحلو لنا أن نلتقي ف...,المساء,مَسَاء,DNo,مَسَاء,سوء
7,Maroc-Primaire-3.xml,1,أبر والدي,2,كان الوقت مساء، وفي المساء يحلو لنا أن نلتقي ف...,يحلو,حَلُوَ,VeP,يَحْلُو,حلو
8,Maroc-Primaire-3.xml,1,أبر والدي,2,كان الوقت مساء، وفي المساء يحلو لنا أن نلتقي ف...,لنا,لِي,PCo,لَنَا,-
9,Maroc-Primaire-3.xml,1,أبر والدي,2,كان الوقت مساء، وفي المساء يحلو لنا أن نلتقي ف...,أن,أَنْ,PaNs,أَن,-


In [27]:
all_pos = df['pos'].unique()
len(all_pos) #169 unique POS tags


169

In [28]:
all_pos

array(['VePa', 'Sifa.S', 'VePa nasikh', 'DNo', 'No NounInfinit',
       'Conj P.Pr', 'VeP', 'PCo', 'PaNs', 'Pr', 'Sifa', 'No.S',
       'Prep P.VeP', 'DSifa', 'Conj P.VeP', 'NoDP', 'Conj P.DNo',
       'Conj P.DSifa', 'Prep P.DNo', 'Alam', 'Co', 'VePa.O', 'Dz',
       'Conj P.No', 'Conj P.VePa', 'VeP.S', 'Ex', 'No.S Pronom', 'Ns',
       'Dz.S', 'La', 'VeP.O', 'Conj P.VeIm', 'No', 'VeIm.O',
       'Conj. P.NoRP', 'VePa.S.O', 'Conj P.Dz', 'Prep P.No',
       'Conj P.No.S', 'Conj P.VePa.O', 'VePa.S', 'Interj', 'PNa',
       'Conj P.PaNs', 'Conj. P.NoDP.S', 'Prep P.Ns', 'Conj.Prep. P.P.No',
       'Conj P.PNa', 'Prep P.No.S', 'PNa.S', 'Prep P.NoRP', 'DNoRP',
       'InteP.No P.No', 'Conj P.No Pronom', 'NoRP', 'Compa', 'DNo Compa',
       'No Pronom', 'Conj P.PCo', 'P.VeP.S.O nasikh', 'Conj P.VeIm.S.O',
       'Conj P.VeP.S.O', 'Conj P.NoDP', 'Conj P.Ns', 'InteP.No.S P.No.S',
       'Conj P.VeIm.S', 'Prep P.Sifa', 'CJ', 'Conj P.Sifa',
       'Conj P.VePa.S', 'Conj P.VePa nasikh', 'InteP.Ve

In [29]:
# Group by POS tags and get corresponding unique words for each tag
pos_word_mapping = df.groupby('pos')['word'].unique().to_dict()

# Print each unique POS tag and the corresponding words
for pos, words in pos_word_mapping.items():
    print(f"POS Tag: {pos}")
    print(f"Words: {', '.join(words)}\n")

POS Tag: Abrv
Words: بي, ان

POS Tag: AbrvF
Words: تكون, طال, يأتي, تلبسه, أليس

POS Tag: Alam
Words: هشام, خالد, ليلى, طيب, أحمد, علي, خاطر, سمير, محمد, مريم, وجدي, ماهر, كريم, حسن, خديجة, قاسم, كم, الفرنسيين, عماد, حسان, عمر, فاطمة, إبراهيم, نبيل, منى, فريد, مبارك, سعيد, فهمي, إيمان, مشرقة, الكسكس, بثينة, رمضان, سعاد, أكتوبر, يناير, مارس, حافظ, سعد, هند, مروان, بلادي, زينب, شعيب, أسامة, أولد, يونيو, إحسان, ليث, بغداد, يونس, شيماء, ديك, صبرا, بحر, طارق, صالح, حليمة, حسام, رضوان, إدريس, رفيق, بدر, كبرت, نجيب, سميرة, ياسر, مطر, غزير, عنان, أحلام, سفيان, الأرز, مسعود, سيد, حاتم, حنون, أمينة, موسى, متى, صفر, ضياء, كونا, أوفي, ظبي, عاشوراء, رضا, وسيم, عنترة, بن, شداد, ياسين, مصطفى, سناء, سلمى, حساما, سميرا, هشاما, نبيلا, بدرا, حمزة, فائز, المصطفى, يوسف, مها, مالي, عثمان, أسعد, عدنان, عصام, ريم, عصفور, كعب, عائشة, فتحي, وكن, هدى, سلوى, كمال, سنتيمترا, غسان, سُعَادُ, مُصْطَفَى, لَيْلَى, رَشِيدَةُ, خَدِيجَةُ, زَكِيَّة, زَكِّيَّةُ, زَكِيَّةُ, يُوسُفُ, سُعَادَ, زَيْنَبَ, خَدِيجَةَ, سَمِيرٌ, نَب

# **Entity Extraction: Test & Compare**

**Entity Extraction NER: Testing two models**

We first test the CAMeLBERT-Mix NER Model on sample sentences and without finetuning

In [167]:
from transformers import pipeline
import torch

In [168]:
def setup_arabic_ner():

    tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-mix-ner", model_max_length=512) # Load tokenizer
    model = AutoModelForTokenClassification.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-mix-ner")  # Load model

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # CPU/GPU
    model = model.to(device)

    # NER pipeline
    ner_pipeline = pipeline("ner", model=model,tokenizer=tokenizer,device=device,aggregation_strategy="simple")

    return ner_pipeline

In [169]:
def extract_entities(text, ner_pipeline):

    try:
        entities = ner_pipeline(text)

        processed_entities = []
        for entity in entities:
            processed_entities.append({
                'entity': entity['word'],
                'label': entity['entity_group'],
                'score': entity['score'],
                'start': entity['start'],
                'end': entity['end']})

        return processed_entities

    except Exception as e:
        print(f"Error processing text: {str(e)}")
        return []

In [170]:
def analyze_arabic_text(text):

    ner_pipeline = setup_arabic_ner()
    entities = extract_entities(text, ner_pipeline)

    # Group entities by type
    entity_groups = {}
    for entity in entities:
        entity_type = entity['label']
        if entity_type not in entity_groups:
            entity_groups[entity_type] = []
        entity_groups[entity_type].append({
            'text': entity['entity'],
            'score': entity['score'],
            'position': (entity['start'], entity['end'])})

    return entity_groups

In [171]:
sentence = df['sentence'][28228]
print(sentence)

وَلَمَّا خَبَزَتِ الْعَجِينَ، عَادَتْ إِلَى صَاحِبَيْهَا وَقَالَتْ لَهُمَا : «لَقَدْ خَبَزْتُ الْعَجِينَ ، فَمَنْ يَأْكُلُهُ؟» فَقَالَ الدِّيكُ الرُّومِيُّ : «أَنَا آكُلُ الْخُبْزَ». وَقَالَتِ الْبَطَّةُ : «أَنَا آكُلُ الْخُبْزَ» فَقَالَتِ الدَّجَاجَةُ : «لَنْ تَأْكُلَا الْخُبْزَ، لِأَنَّكُمَا لَا تَشْتَغِلَانِ». وَذَهَبَتِ الدَّجَاجَةُ الشَّغَّالَةُ، وَأَكَلَتِ الْخُبْزَ مَعَ فِرَاخِهَا الصِّغَارِ.


The model was trained on sentences without diacritics, so we remove them

In [172]:
import re

In [173]:
def remove_diacritics(text):
    arabic_diacritics = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    return re.sub(arabic_diacritics, '', text)

In [174]:
#test1
mosar_sentence_wd = remove_diacritics(sentence)
print("Sentence without Diacritics:", mosar_sentence_wd)

Sentence without Diacritics: ولما خبزت العجين، عادت إلى صاحبيها وقالت لهما : «لقد خبزت العجين ، فمن يأكله؟» فقال الديك الرومي : «أنا آكل الخبز». وقالت البطة : «أنا آكل الخبز» فقالت الدجاجة : «لن تأكلا الخبز، لأنكما لا تشتغلان». وذهبت الدجاجة الشغالة، وأكلت الخبز مع فراخها الصغار.


In [175]:
entities = analyze_arabic_text(mosar_sentence_wd)
print(entities) #empty list -> the model did not recognize the entities

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


{}


In [176]:
mosar_sentence_2 = df['sentence'][150]
print(mosar_sentence_2)

وقالت ليلى : ((لقد أوجب علينا ربنا، سبحانه وتعالى، أن نخاطب اباءنا وأمهاتنا باحترام وبكلام طيب)).


In [177]:
entities_2 = analyze_arabic_text(mosar_sentence_2)
print(entities_2) #PERSON entity detected with 96% accuracy

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


{'PERS': [{'text': 'ليلى', 'score': 0.96104264, 'position': (6, 10)}]}


In [178]:
story_text = "إمارة أبوظبي هي إحدى إمارات دولة الإمارات العربية المتحدة السبع" #a sentence not in the Mosar dataset
results = analyze_arabic_text(story_text)

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [179]:
print(results) #high accuracy in all the entities

{'LOC': [{'text': 'أبوظبي', 'score': 0.9928443, 'position': (6, 12)}, {'text': 'الإمارات العربية المتحدة', 'score': 0.95342207, 'position': (33, 57)}]}


In the same way, we test the "hatmimoha/arabic-ner" model now

In [180]:
def setup_arabic_ner_2():

    tokenizer = AutoTokenizer.from_pretrained("hatmimoha/arabic-ner", model_max_length=512) # Load tokenizer
    model = AutoModelForTokenClassification.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-mix-ner")  # Load model

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # CPU/GPU
    model = model.to(device)

    # NER pipeline
    ner_pipeline = pipeline("ner", model=model,tokenizer=tokenizer,device=device,aggregation_strategy="simple")

    return ner_pipeline

In [181]:
def extract_entities_2(text, ner_pipeline):

    try:
        entities = ner_pipeline(text)

        processed_entities = []
        for entity in entities:
            processed_entities.append({
                'entity': entity['word'],
                'label': entity['entity_group'],
                'score': entity['score'],
                'start': entity['start'],
                'end': entity['end']})

        return processed_entities

    except Exception as e:
        print(f"Error processing text: {str(e)}")
        return []

In [182]:
def analyze_arabic_text_2(text):

    ner_pipeline = setup_arabic_ner()
    entities = extract_entities(text, ner_pipeline)

    # Group entities by type
    entity_groups = {}
    for entity in entities:
        entity_type = entity['label']
        if entity_type not in entity_groups:
            entity_groups[entity_type] = []
        entity_groups[entity_type].append({
            'text': entity['entity'],
            'score': entity['score'],
            'position': (entity['start'], entity['end'])})

    return entity_groups

In [183]:
entities_3 = analyze_arabic_text_2(mosar_sentence_2)
print(entities_3) #96% accuracy in identifying the PERSON

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


{'PERS': [{'text': 'ليلى', 'score': 0.96104264, 'position': (6, 10)}]}


In [184]:
entities_4 = analyze_arabic_text_2(mosar_sentence_wd)
print(entities_4) #empty tokens

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


{}


In [185]:
entities_5 = analyze_arabic_text_2(story_text)
print(entities_5)  #all entities identified

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


{'LOC': [{'text': 'أبوظبي', 'score': 0.9928443, 'position': (6, 12)}, {'text': 'الإمارات العربية المتحدة', 'score': 0.95342207, 'position': (33, 57)}]}


# **Data preparation for Annotation**




First Fine-tuning: We choose the model to fine tune, and use the annotated entities in the first_annotation.jsonl file

In [49]:
ds = load_dataset("asas-ai/ANERCorp")
print(ds)

README.md:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

(…)-00000-of-00001-83c5047e14e68965.parquet:   0%|          | 0.00/855k [00:00<?, ?B/s]

(…)-00000-of-00001-245173671c05c71a.parquet:   0%|          | 0.00/175k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/125102 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25008 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['word', 'tag'],
        num_rows: 125102
    })
    test: Dataset({
        features: ['word', 'tag'],
        num_rows: 25008
    })
})


In [50]:
print(ds['train'][0])

{'word': 'فرانكفورت', 'tag': 'B-LOC'}


**The Doccano Dataset Creation**


The model was fine-tuned on ANERcorp, the input data for fine-tuning should follow the same formatting as the ANERcorp dataset

The ANERCorp is organized into words and their corresponding tags. With tags following the BIO (Begin-Inside-Outside) tagging scheme. We follow this same format to manually annotate our sample data.

The sample data is annotated using the Doccano software. The initial results are stores in the first_annotation.jsonl file "".

In [51]:
import os
import json
import re
from pathlib import Path

In [52]:
def clean_and_split_sentences(text):
    text = re.sub(r'\d+-', '', text) # to remove the title numbers and dashes

    sentences = []
    current = ""

    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Handle each character
        for char in line:
            current += char
            if char in ['.', '!', '؟'] and not current.strip().startswith('"'):
                if current.strip():
                    # Clean up the sentence
                    clean_sent = current.strip()
                    # Remove any remaining numbers at start
                    clean_sent = re.sub(r'^\d+\s*', '', clean_sent)
                    if clean_sent:
                        sentences.append(clean_sent)
                current = ""

        # Handle line endings without punctuation
        if current.strip():
            sentences.append(current.strip())
            current = ""

    return sentences

In [53]:
def process_directory(root_dir):
    root_path = Path(root_dir)
    doccano_format = []

    for grade_folder in root_path.iterdir():
        if grade_folder.is_dir():
            print(f"Processing grade folder: {grade_folder.name}")

            for txt_file in grade_folder.glob('*.txt'):
                print(f"  Processing file: {txt_file.name}")

                try:
                    with open(txt_file, 'r', encoding='utf-8') as f:
                        text = f.read()

                    sentences = clean_and_split_sentences(text)

                    for sentence in sentences:
                        if sentence.strip():
                            # Include text field and metadata for source document
                            doccano_format.append({
                                "text": sentence.strip(),
                                "entities": [],
                                "metadata": {
                                    "source_document": txt_file.name,
                                    "grade": grade_folder.name
                                }
                            })

                except Exception as e:
                    print(f"    Error processing {txt_file}: {str(e)}")

    return doccano_format

In [54]:
def save_doccano_file(sentences, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for sentence in sentences:
            f.write(json.dumps(sentence, ensure_ascii=False) + '\n')

In [55]:
root_directory = "/content/drive/MyDrive/AKG Research/RawText_1to3"
output_file = "doccano_ready.jsonl"

print(f"Starting to process files in {root_directory}")
sentences = process_directory(root_directory)

print(f"\nFound {len(sentences)} sentences")
save_doccano_file(sentences, output_file)
print(f"Output saved to {output_file}")

# Print first few lines to verify format
print("\nFirst 3 lines of output:")
with open(output_file, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 3:
            break
        print(f"\nLine {i+1}:")
        print(json.dumps(json.loads(line), ensure_ascii=False, indent=2))

Starting to process files in /content/drive/MyDrive/AKG Research/RawText_1to3
Processing grade folder: 3
  Processing file: حكاية الفصول الأربعة.txt
  Processing file: غذاء المريض.txt
  Processing file: المهندس الصغير.txt
  Processing file: صندوق العروسة.txt
  Processing file: نتهيأ للإفطار.txt
  Processing file: إلى بلاد الإسكيمو.txt
  Processing file: الطيارة.txt
  Processing file: الكرة.txt
  Processing file: كن صديقي.txt
  Processing file: مهن الفلاح.txt
  Processing file: يا وطني.txt
  Processing file: إني الماء.txt
  Processing file: الشرطي في خدمتنا.txt
  Processing file: الحمامة و النملة.txt
  Processing file: التعاونية الناجحة.txt
  Processing file: أطفئ النور ونم.txt
  Processing file: أساعد غيري.txt
  Processing file: احسب معي.txt
  Processing file: أحافظ على نظافة مدرستي.txt
  Processing file: أبر والدي.txt
  Processing file: الفواكه و الصحة.txt
  Processing file: خديجة تتعلم.txt
  Processing file: حزوره.txt
  Processing file: جزيرتي الملونة.txt
  Processing file

The data is now ready, stored as doccano_ready.jsonl and ready to be imported to Doccano

# **Manual Annotation**

Was done using the Doccano Software: https://github.com/doccano/doccano

# **Data Preparation for Fine-tuning**

In [120]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=093d7d14b2035d32263c212556e71a5d1b81a746c5d164580c8f58862460dfe5
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [138]:
import os
import json
import random
from typing import List, Dict
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import Dataset
import torch
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score


We Convert Doccano JSONL to CoNLL Format

In [88]:
# Load the annotations
file_path = "/content/drive/MyDrive/AKG Research/latest_annotations.jsonl"
with open(file_path, 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

In [89]:
print(data[0])

{'id': 14996, 'text': 'نهض عادل من نومه، فرأى زهرته التي اعتنى بها وسقاها، تذبل وتذبل، حزن عادل لما رأى لونها يشحب، ووريقاتها تتجعد، فقال له أبوه : (( لا تحزن يا ولدي، فقد انتهى زمان زهرتك، لأن لكل زهرة عمرا، ولكل فصل من فصول السنة أزهارا وثمارا)).', 'label': [[0, 3, 'O'], [4, 8, 'B-PERS'], [9, 11, 'O'], [12, 16, 'O'], [18, 22, 'O'], [23, 28, 'B-OBJ'], [29, 33, 'O'], [34, 39, 'O'], [40, 43, 'O'], [44, 50, 'O'], [52, 56, 'O'], [57, 62, 'O'], [64, 67, 'O'], [68, 72, 'B-PERS'], [73, 76, 'O'], [77, 80, 'O'], [81, 86, 'B-OBJ'], [87, 91, 'O'], [93, 102, 'O'], [103, 108, 'O'], [110, 114, 'O'], [115, 117, 'O'], [118, 122, 'O'], [128, 130, 'O'], [131, 135, 'O'], [136, 138, 'O'], [139, 143, 'O'], [145, 148, 'O'], [149, 154, 'O'], [155, 159, 'O'], [160, 165, 'O'], [167, 170, 'O'], [171, 174, 'O'], [175, 179, 'O'], [180, 184, 'O'], [186, 190, 'O'], [191, 194, 'O'], [195, 197, 'O'], [198, 202, 'O'], [203, 208, 'O'], [209, 215, 'O'], [216, 222, 'O']], 'metadata': {'source_document': 'حكاية الفصول ال

In [93]:
def read_jsonl(file_path: str) -> List[Dict]:
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

In [94]:
def convert_to_conll_format(annotation: Dict) -> List[str]:
    text = annotation['text']
    labels = sorted(annotation['label'], key=lambda x: x[0])

    conll_lines = []
    for start, end, label in labels:
        word = text[start:end]
        words = word.split()
        for w in words:
            conll_lines.append(f"{w} {label}")
    return conll_lines

In [95]:
def split_data(data: List[Dict], train_ratio=0.8, dev_ratio=0.1, test_ratio=0.1) -> tuple:
    random.shuffle(data)
    n = len(data)
    train_end = int(n * train_ratio)
    dev_end = int(n * (train_ratio + dev_ratio))

    return (
        data[:train_end],
        data[train_end:dev_end],
        data[dev_end:]
    )

In [96]:
def save_conll_file(data: List[Dict], output_file: str):
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write('-DOCSTART- -X- -X- O\n\n')  # Standard CoNLL-2003 header

        for annotation in data:
            conll_lines = convert_to_conll_format(annotation)
            for line in conll_lines:
                f.write(line + '\n')
            f.write('\n')

In [97]:
def extract_unique_labels(data: List[Dict]) -> set:
    labels = set()
    for annotation in data:
        for _, _, label in annotation['label']:
            labels.add(label)
    return labels

In [98]:
input_file = "/content/drive/MyDrive/AKG Research/latest_annotations.jsonl"
output_dir = "ner_data"
os.makedirs(output_dir, exist_ok=True)

data = read_jsonl(input_file)
train_data, dev_data, test_data = split_data(data)

save_conll_file(train_data, os.path.join(output_dir, "train.txt"))
save_conll_file(dev_data, os.path.join(output_dir, "dev.txt"))
save_conll_file(test_data, os.path.join(output_dir, "test.txt"))

labels = extract_unique_labels(data)
with open(os.path.join(output_dir, "labels.txt"), 'w', encoding='utf-8') as f:
    for label in sorted(labels):
        f.write(f"{label}\n")

We finetune the model

In [100]:
# We set up the model and tokenizer
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-mix-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [129]:
def read_conll_file(file_path):
    texts, tags = [], []
    current_words, current_tags = [], []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()

            if line.startswith('-DOCSTART-') or not line:
                if current_words:
                    texts.append(current_words)
                    tags.append(current_tags)
                    current_words, current_tags = [], []
            else:
                splits = line.split(' ')
                word = splits[0]
                tag = splits[-1]
                current_words.append(word)
                current_tags.append(tag)

    if current_words:  # Don't forget last sentence
        texts.append(current_words)
        tags.append(current_tags)

    return texts, tags

In [149]:
# we read the unique labels we saved to labels.txt
# Read labels
with open("/content/ner_data/labels.txt", "r", encoding="utf-8") as f:
    label_list = [line.strip() for line in f]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

In [150]:
print(label2id)

{'B-LOC': 0, 'B-OBJ': 1, 'B-PERS': 2, 'B-TIME': 3, 'I-LOC': 4, 'I-PERS': 5, 'I-TIME': 6, 'O': 7}


In [151]:
print(id2label)

{0: 'B-LOC', 1: 'B-OBJ', 2: 'B-PERS', 3: 'B-TIME', 4: 'I-LOC', 5: 'I-PERS', 6: 'I-TIME', 7: 'O'}


In [152]:
# Load datasets
train_texts, train_tags = read_conll_file("ner_data/train.txt")
dev_texts, dev_tags = read_conll_file("ner_data/dev.txt")
test_texts, test_tags = read_conll_file("ner_data/test.txt")

In [153]:
# Convert to Dataset objects
def create_dataset(texts, tags):
    return Dataset.from_dict({
        "tokens": texts,
        "ner_tags": [[label2id[tag] for tag in doc] for doc in tags]
    })

In [154]:
train_dataset = create_dataset(train_texts, train_tags)
dev_dataset = create_dataset(dev_texts, dev_tags)
test_dataset = create_dataset(test_texts, test_tags)

In [155]:
# Tokenization function
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding=True,
        max_length=128  # Adjust this based on your needs
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [156]:
# Tokenize datasets
train_tokenized = train_dataset.map(tokenize_and_align_labels, batched=True)
dev_tokenized = dev_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized = test_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [157]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [158]:
def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return {
        "accuracy": accuracy_score(true_labels, true_predictions),
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions)
    }

In [163]:
# Training arguments
training_args = TrainingArguments(
    output_dir="camelbert_finetuned",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none",  # I chose to disable wandb
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [164]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=dev_tokenized,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [165]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.620357,0.832536,0.200000,0.033333,0.057143
2,No log,0.572721,0.832536,0.250000,0.066667,0.105263
3,No log,0.510473,0.842105,0.416667,0.166667,0.238095
4,No log,0.481112,0.856459,0.615385,0.266667,0.372093
5,No log,0.457800,0.861244,0.529412,0.300000,0.382979
6,No log,0.433115,0.866029,0.526316,0.333333,0.408163
7,No log,0.448403,0.870813,0.588235,0.333333,0.425532
8,No log,0.439457,0.875598,0.611111,0.366667,0.458333
9,No log,0.430591,0.880383,0.619048,0.433333,0.509804
10,No log,0.429342,0.880383,0.590909,0.433333,0.500000


TrainOutput(global_step=50, training_loss=0.23497228622436522, metrics={'train_runtime': 1583.0217, 'train_samples_per_second': 0.505, 'train_steps_per_second': 0.032, 'total_flos': 35113973788800.0, 'train_loss': 0.23497228622436522, 'epoch': 10.0})

In [166]:
test_results = trainer.evaluate(test_tokenized)
print("\nTest Results:", test_results)


Test Results: {'eval_loss': 0.44346582889556885, 'eval_accuracy': 0.8655913978494624, 'eval_precision': 0.7407407407407407, 'eval_recall': 0.625, 'eval_f1': 0.6779661016949152, 'eval_runtime': 2.2051, 'eval_samples_per_second': 7.256, 'eval_steps_per_second': 0.454, 'epoch': 10.0}


# **Trial and Error**

**Few Shot Learning**

In [61]:
tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-mix-ner")
model = AutoModelForTokenClassification.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-mix-ner")

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [62]:
support_sets = {
    'PERS': ["خَدِيجَةَ", "سُعَادَ", "سَمِيرٌ", "سلمى", "أحلام"],
    'LOC': ["غابة","هناك","هنا","يساراً","يميناً","الْأَسِرَّةِ","الرَّمْلِ", "المدرسة", "الرصيف", "السوق"]
}

In [63]:
def encode_words(words):
    inputs = tokenizer(words, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_state = outputs[0]
    mask = inputs['attention_mask'].unsqueeze(-1).expand(last_hidden_state.size()).float()
    sum_embeddings = torch.sum(last_hidden_state * mask, 1)
    sum_mask = torch.clamp(mask.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [64]:
support_sets.items()

dict_items([('PERS', ['خَدِيجَةَ', 'سُعَادَ', 'سَمِيرٌ', 'سلمى', 'أحلام']), ('LOC', ['غابة', 'هناك', 'هنا', 'يساراً', 'يميناً', 'الْأَسِرَّةِ', 'الرَّمْلِ', 'المدرسة', 'الرصيف', 'السوق'])])

In [65]:
encoded_support_sets = {label: encode_words(words) for label, words in support_sets.items()}

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [66]:
def predict_entity(word):
    # Encode the input word
    encoded_word = encode_words([word])
    similarities = {}
    for label, encoded_support in encoded_support_sets.items():
        similarity = cosine_similarity(encoded_word.numpy(), encoded_support.numpy())[0].mean()
        similarities[label] = similarity
    predicted_label = max(similarities, key=similarities.get)

    return predicted_label if similarities[predicted_label] > 0.5 else 'O'

In [67]:
def predict_sentence(sentence):
    words = sentence.split()
    predictions = []
    for i, word in enumerate(words):
        prediction = predict_entity(word)
        if prediction != 'O':
            # Check if it's the start of an entity
            if i == 0 or predictions[-1][1] != prediction:
                predictions.append((word, f'B-{prediction}'))
            else:
                predictions.append((word, f'I-{prediction}'))
        else:
            predictions.append((word, 'O'))
    return predictions

In [68]:
from sklearn.metrics.pairwise import cosine_similarity

In [69]:
test_words = ["محمد", "القاهرة", "كتاب", "دمشق", "فاطمة", "مكتبة", "علي", "بيروت"]
print("Individual word predictions:")
for word in test_words:
    prediction = predict_entity(word)
    print(f"Word: {word}, Predicted Entity: {prediction}")

Individual word predictions:
Word: محمد, Predicted Entity: PERS
Word: القاهرة, Predicted Entity: LOC
Word: كتاب, Predicted Entity: LOC
Word: دمشق, Predicted Entity: LOC
Word: فاطمة, Predicted Entity: PERS
Word: مكتبة, Predicted Entity: LOC
Word: علي, Predicted Entity: PERS
Word: بيروت, Predicted Entity: LOC


In [70]:
text_2 = "كان الوقت مساء، وفي المساء يحلو لنا أن نلتقي في منزل صديق من أصدقائنا، لننجز واجباتنا المدرسية، ونراجع دروسنا. كان الواجب هذه المرة التدرب على كتابة الايتين الثالثة والعشرين والرابعة والعشرين من سورة الإسراء بالرسم العثماني."

print("Sentence prediction for text_2:")
result = predict_sentence(text_2)
for word, label in result:
    print(f"{word}: {label}")

Sentence prediction for text_2:
كان: B-LOC
الوقت: B-LOC
مساء،: B-LOC
وفي: B-LOC
المساء: B-LOC
يحلو: B-LOC
لنا: B-LOC
أن: B-LOC
نلتقي: B-LOC
في: B-LOC
منزل: B-LOC
صديق: B-LOC
من: B-LOC
أصدقائنا،: B-LOC
لننجز: B-LOC
واجباتنا: B-LOC
المدرسية،: B-LOC
ونراجع: B-LOC
دروسنا.: B-LOC
كان: B-LOC
الواجب: B-LOC
هذه: B-LOC
المرة: B-LOC
التدرب: B-LOC
على: B-LOC
كتابة: B-LOC
الايتين: B-LOC
الثالثة: B-LOC
والعشرين: B-LOC
والرابعة: B-LOC
والعشرين: B-LOC
من: B-LOC
سورة: B-LOC
الإسراء: B-LOC
بالرسم: B-LOC
العثماني.: B-PERS


**Prototypical Networks for Few-Shot NER**

In [71]:
def load_jsonl_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [72]:
jsonl_path = "/content/drive/MyDrive/AKG Research/SupportSet.jsonl" #obtained from Doccano
data = load_jsonl_data(jsonl_path)

In [73]:
data[0]

{'id': 4,
 'text': 'يُعْجِبُ خَدِيجَةَ وَسُعَادَ أَنْ تَلْعَبَا الْمُمَرِّضَةَ، فَجَمِيعُ الدُّمَى وَالْحَيَوَانَاتِ مَرِيضَةٌ وَعَلَى الْأَسِرَّةِ.',
 'label': [[0, 8, 'OTHER'],
  [9, 18, 'PERSON'],
  [18, 21, 'OTHER'],
  [21, 28, 'PERSON'],
  [29, 33, 'OTHER'],
  [34, 43, 'OTHER'],
  [44, 58, 'OTHER'],
  [60, 69, 'OTHER'],
  [70, 78, 'OTHER'],
  [78, 81, 'OTHER'],
  [81, 96, 'OTHER'],
  [97, 106, 'OTHER'],
  [107, 109, 'OTHER'],
  [109, 114, 'OTHER'],
  [115, 127, 'LOCATION']],
 'Comments': []}

In [74]:
label2id = {'PERSON':0, 'TIME': 1, 'LOCATION': 2, 'OTHER': 3}

In [75]:
def tokenize_and_align_labels(data, tokenizer, label2id, max_length=128):
    tokenized_inputs = tokenizer(
        data['text'], padding='max_length', truncation=True,
        max_length=max_length, return_offsets_mapping=True
    )
    labels = [-100] * len(tokenized_inputs['input_ids'])  # Initialize ignored labels

    for entity in data['label']:
        start, end, label = entity
        for idx, (offset_start, offset_end) in enumerate(tokenized_inputs['offset_mapping']):
            if offset_start >= start and offset_end <= end:
                labels[idx] = label2id[label]

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

In [76]:
def process_dataset(data, tokenizer, label2id):
    processed_data = []
    for sample in data:
        processed_sample = tokenize_and_align_labels(sample, tokenizer, label2id)
        processed_data.append(processed_sample)
    return processed_data


In [77]:
processed_data = process_dataset(data, tokenizer, label2id)

In [78]:
class ArabicNERDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.data[idx]['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(self.data[idx]['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(self.data[idx]['labels'], dtype=torch.long)
        }

In [79]:
# Create the DataLoader
dataset = ArabicNERDataset(processed_data)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [80]:
class PrototypicalNetwork(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)

    def forward(self, support_batch, query_batch):
        support_embeddings = self.encode_set(support_batch)
        query_embeddings = self.encode(query_batch['input_ids'], query_batch['attention_mask'])

        prototypes = support_embeddings.mean(dim=0, keepdim=True)

        distances = self.compute_distances(query_embeddings, prototypes)
        return -distances

    def encode(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state[:, 0, :]

    def encode_set(self, support_batch):
        input_ids = support_batch['input_ids']
        attention_mask = support_batch['attention_mask']
        return self.encode(input_ids, attention_mask)

    def compute_distances(self, query_embeddings, prototypes):
        n_queries = query_embeddings.size(0)
        n_prototypes = prototypes.size(0)

        query_embeddings = query_embeddings.unsqueeze(1).expand(n_queries, n_prototypes, -1)
        prototypes = prototypes.expand(n_queries, n_prototypes, -1)

        return torch.sum((query_embeddings - prototypes) ** 2, dim=-1)


In [81]:
tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-mix-ner")
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-mix-ner"

In [82]:
model = PrototypicalNetwork(model_name)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [83]:
for epoch in range(5):
    model.train()
    for support_batch in dataloader:
        query_batch = support_batch  # Same batch for simplicity

        loss = model(support_batch, query_batch)

        optimizer.zero_grad()
        loss.mean().backward()
        optimizer.step()

        print(f"Epoch {epoch + 1}, Loss: {loss.mean().item()}")

Epoch 1, Loss: -111.71559143066406
Epoch 1, Loss: -117.83995056152344
Epoch 1, Loss: -135.7628173828125
Epoch 1, Loss: -177.5293731689453
Epoch 1, Loss: -225.57540893554688
Epoch 2, Loss: -221.981201171875
Epoch 2, Loss: -218.37411499023438
Epoch 2, Loss: -212.78414916992188
Epoch 2, Loss: -368.99462890625
Epoch 2, Loss: -341.6263427734375
Epoch 3, Loss: -392.9284973144531
Epoch 3, Loss: -404.94281005859375
Epoch 3, Loss: -441.6446533203125
Epoch 3, Loss: -283.7538146972656
Epoch 3, Loss: -413.59130859375
Epoch 4, Loss: -465.2208251953125
Epoch 4, Loss: -423.9948425292969
Epoch 4, Loss: -462.04718017578125
Epoch 4, Loss: -378.0181884765625
Epoch 4, Loss: -496.24285888671875
Epoch 5, Loss: -419.20208740234375
Epoch 5, Loss: -484.5773620605469
Epoch 5, Loss: -458.0325927734375
Epoch 5, Loss: -507.7066650390625
Epoch 5, Loss: -505.225830078125


In [84]:
model.eval()
with torch.no_grad():
    for query_batch in dataloader:
        distances = model(query_batch, query_batch)  # Shape: [batch_size, num_classes]
        print(f"Distances:\n{distances}")

        # Get the predicted class for each query in the batch
        predicted_classes = torch.argmin(distances, dim=-1)  # Shape: [batch_size]

        # Print predictions for all queries in the batch
        for i, predicted_class in enumerate(predicted_classes):
            print(f"Query {i}: Predicted Class = {predicted_class.item()}")

Distances:
tensor([[-278.9778],
        [-650.9176],
        [-277.8119],
        [-451.0616]])
Query 0: Predicted Class = 0
Query 1: Predicted Class = 0
Query 2: Predicted Class = 0
Query 3: Predicted Class = 0
Distances:
tensor([[-310.5333],
        [-739.4657],
        [-293.5261],
        [-370.9735]])
Query 0: Predicted Class = 0
Query 1: Predicted Class = 0
Query 2: Predicted Class = 0
Query 3: Predicted Class = 0
Distances:
tensor([[-196.1452],
        [-158.4338],
        [-173.4148],
        [-646.8871]])
Query 0: Predicted Class = 0
Query 1: Predicted Class = 0
Query 2: Predicted Class = 0
Query 3: Predicted Class = 0
Distances:
tensor([[-453.5782],
        [-399.1483],
        [-319.1356],
        [-328.4258]])
Query 0: Predicted Class = 0
Query 1: Predicted Class = 0
Query 2: Predicted Class = 0
Query 3: Predicted Class = 0
Distances:
tensor([[-354.1573],
        [-356.3444],
        [-569.8266],
        [-546.5502]])
Query 0: Predicted Class = 0
Query 1: Predicted Class = 

In [85]:
# Custom sentences to test the model
test_sentences = [
    "زار محمد القاهرة صباحاً.",  # Person, Location, Time
    "سلمى ذهبت إلى المدرسة.",  # Person, Location
    "أحمد يحب السباحة في الصباح.",  # Person, Time
    "ذهبنا إلى الدار البيضاء في المساء."  # Location, Time
]


In [86]:
def encode_sentence(sentence, tokenizer, max_length=128):
    """Tokenize and encode a sentence."""
    encoding = tokenizer(
        sentence,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )
    return encoding['input_ids'], encoding['attention_mask']


In [87]:
import torch.nn.functional as F

# Perform inference on the custom sentences
model.eval()
with torch.no_grad():
    for sentence in test_sentences:
        input_ids, attention_mask = encode_sentence(sentence, tokenizer)

        # Forward pass to get distances to prototypes
        query_batch = {'input_ids': input_ids, 'attention_mask': attention_mask}
        distances = model(query_batch, query_batch)  # [1, num_classes]

        # Convert distances to probabilities
        probs = F.softmax(-distances, dim=-1)  # [1, num_classes]

        # Get the predicted class
        predicted_class = torch.argmax(probs, dim=-1).item()

        print(f"Sentence: {sentence}")
        print(f"Predicted Class: {predicted_class}\n")


Sentence: زار محمد القاهرة صباحاً.
Predicted Class: 0

Sentence: سلمى ذهبت إلى المدرسة.
Predicted Class: 0

Sentence: أحمد يحب السباحة في الصباح.
Predicted Class: 0

Sentence: ذهبنا إلى الدار البيضاء في المساء.
Predicted Class: 0

